### **Búsqueda y Minería de Información 2021-22**
### Universidad Autónoma de Madrid, Escuela Politécnica Superior
### Grado en Ingeniería Informática, 4º curso
# **Motores de búsqueda e indexación**

Fechas:

* Comienzo: lunes 21 / martes 22 de febrero
* Entrega: lunes 28 / martes 29 de marzo (14:00)

# Introducción

## Objetivos

Los objetivos de esta práctica son:

* La implementación eficiente de funciones de ránking, particularizada en el modelo vectorial.
*	La implementación de índices eficientes para motores de búsqueda. 
*	La implementación de un método de búsqueda proximal.
*	La dotación de estructuras de índice posicional que soporten la búsqueda proximal.
*	La implementación del algoritmo PageRank.

Se desarrollarán implementaciones de índices utilizando un diccionario y listas de postings. Y se implementará el modelo vectorial utilizando estas estructuras más eficientes para la ejecución de consultas.

Los ejercicios básicos consistirán en la implementación de algoritmos y técnicas estudiados en las clases de teoría, con algunas propuestas de extensión opcionales. Se podrá comparar el rendimiento de las diferentes versiones de índices y buscadores, contrastando la coherencia con los planteamientos estudiados a nivel teórico.

Mediante el nivel de abstracción seguido, se conseguirán versiones intercambiables de índices y buscadores. El único buscador que no será intercambiable es el de Whoosh, que sólo funcionará con sus propios índices.

## Material proporcionado

Se proporcionan (bien en el curso de Moodle o dentro de este documento):

*	Varias clases e interfaces Python a lo largo de este *notebook*, con las que el estudiante integrará las suyas propias. 
Las clases parten del código de la práctica anterior.
Igual que en la práctica 1, la función **main** implementa un programa que deberá funcionar con las clases a implementar por el estudiante.
*	Las colecciones de prueba de la práctica 1: <ins>toys.zip</ins> (que se descomprime en dos carpetas toy1 y toy2), <ins>docs1k.zip</ins> con 1.000 documentos HTML y un pequeño fichero <ins>urls.txt</ins>. 
*	Una colección más grande: <ins>docs10k.zip</ins> con 10.000 documentos HTML.
*	Varios grafos para probar PageRank: <ins>graphs.zip</ins>.
*	Un documento de texto <ins>output.txt</ins> con la salida estándar que deberá producir la ejecución de la función main.

In [27]:
import os, os.path
import re
import math
import pickle
import zipfile
from abc import ABC, abstractmethod
from urllib.request import urlopen
from bs4 import BeautifulSoup

class Config(object):
  # variables de clase
  NORMS_FILE = "docnorms.dat"
  PATHS_FILE = "docpaths.dat"
  INDEX_FILE = "serialindex.dat"
  DICTIONARY_FILE = "dictionary.dat"
  POSTINGS_FILE = "postings.dat"

class BasicParser:
    @staticmethod
    def parse(text):
        return re.findall(r"[^\W\d_]+|\d+", text.lower())
    
def tf(freq):
    return 1 + math.log2(freq) if freq > 0 else 0

def idf(ndocs_t, ndocs):
    return math.log2((ndocs+1)/(ndocs_t+0.5))

"""
    This is an abstract class for the search engines
"""
class Searcher(ABC):
    def __init__(self, index, parser=BasicParser()):
        self.index = index
        self.parser = parser
    @abstractmethod
    def search(self, query, cutoff):
        """ Returns a list of documents encapsulated in a SearchRanking class """

class Index:
    def __init__(self, dir=None):
        self.docmap = []
        self.modulemap = {}
        if dir: self.open(dir)
    def add_doc(self, path):
        self.docmap.append(path)  # Assumed to come in order
    def doc_path(self, docid):
        return self.docmap[docid]
    def doc_module(self, docid):
        if docid in self.modulemap:
            return self.modulemap[docid]
        return None
    def ndocs(self):
        return len(self.docmap)
    def doc_freq(self, term):
        return len(self.postings(term))
    
    def term_freq(self, term, docID):
        post = self.postings(term)
        if post is None: return 0
        for posting in post:
            if posting[0] == docID:
                return posting[1]
        return 0
    
    def total_freq(self, term):
        freq = 0
        for posting in self.postings(term):
            freq += posting[1]
        return freq
    
    def postings(self, term):
        return list()
    
    def positional_postings(self, term):
        # used in positional implementations
        return list()
    
    def all_terms(self):
        return list()
    
    def save(self, dir):
        if not self.modulemap: self.compute_modules()
        p = os.path.join(dir, Config.NORMS_FILE)
        with open(p, 'wb') as f:
            pickle.dump(self.modulemap, f)        
    def open(self, dir):
        try:
            p = os.path.join(dir, Config.NORMS_FILE)
            with open(p, 'rb') as f:
                self.modulemap = pickle.load(f)
        except OSError:
            # the file may not exist the first time
            pass
    def compute_modules(self):
        for term in self.all_terms():
            idf_score = idf(self.doc_freq(term), self.ndocs())
            post = self.postings(term)
            if post is None: continue
            for docid, freq in post:
                if docid not in self.modulemap: self.modulemap[docid] = 0
                self.modulemap[docid] += math.pow(tf(freq) * idf_score, 2)
        for docid in range(self.ndocs()):
            self.modulemap[docid] = math.sqrt(self.modulemap[docid]) if docid in self.modulemap else 0
        

class Builder:
    def __init__(self, dir, parser=BasicParser()):
        if os.path.exists(dir): shutil.rmtree(dir)
        os.makedirs(dir)
        self.parser = parser
    def build(self, path):
        if zipfile.is_zipfile(path):
            self.index_zip(path)
        elif os.path.isdir(path):
            self.index_dir(path)
        else:
            self.index_url_file(path)
    def index_zip(self, filename):
        file = zipfile.ZipFile(filename, mode='r', compression=zipfile.ZIP_DEFLATED)
        for name in sorted(file.namelist()):
            with file.open(name, "r", force_zip64=True) as f:
                self.index_document(name, BeautifulSoup(f.read().decode("utf-8"), "html.parser").text)
        file.close()
    def index_dir(self, dir):
        for subdir, dirs, files in os.walk(dir):
            for file in sorted(files):
                path = os.path.join(dir, file)
                with open(path, "r") as f:
                    self.index_document(path, f.read())
    def index_url_file(self, file):
        with open(file, "r") as f:
            self.index_urls(line.rstrip('\n') for line in f)
    def index_urls(self, urls):
        for url in urls:
            self.index_document(url, BeautifulSoup(urlopen(url).read().decode("utf-8"), "html.parser").text)
    def index_document(self, path, text):
        pass
    def commit(self):
        pass

In [28]:
# from previous lab
class SlowVSMSearcher(Searcher):
    def __init__(self, index, parser=BasicParser()):
        super().__init__(index, parser)

    def search(self, query, cutoff):
        qterms = self.parser.parse(query)
        ranking = SearchRanking(cutoff)
        for docid in range(self.index.ndocs()):
            score = self.score(docid, qterms)
            if score:
                ranking.push(self.index.doc_path(docid), score)
        return ranking

    def score(self, docid, qterms):
        prod = 0
        for term in qterms:
            prod += tf(self.index.term_freq(term, docid)) \
                    * idf(self.index.doc_freq(term), self.index.ndocs())
        mod = self.index.doc_module(docid)
        if mod:
            return prod / mod
        return 0

In [29]:
try:
  import whoosh
except ModuleNotFoundError:
  !pip install whoosh
  import whoosh
from whoosh.fields import Schema, TEXT, ID
from whoosh.formats import Format
from whoosh.qparser import QueryParser

# A schema in Whoosh is the set of possible fields in a document in
# the search space. We just define a simple 'Document' schema, with
# a path (a URL or local pathname) and a content.
SimpleDocument = Schema(
        path=ID(stored=True),
        content=TEXT(phrase=False))
ForwardDocument = Schema(
        path=ID(stored=True),
        content=TEXT(phrase=False,vector=Format))
PositionalDocument = Schema(
        path=ID(stored=True),
        content=TEXT(phrase=True))

class WhooshBuilder(Builder):
    def __init__(self, dir, schema=SimpleDocument):
        super().__init__(dir)
        self.whoosh_writer = whoosh.index.create_in(dir, schema).writer(procs=1, limitmb=16384, multisegment=True)
        self.dir = dir

    def index_document(self, p, text):
        self.whoosh_writer.add_document(path=p, content=text)

    def commit(self):
        self.whoosh_writer.commit()
        index = WhooshIndex(self.dir)
        index.save(self.dir)

class WhooshForwardBuilder(WhooshBuilder):
    def __init__(self, dir):
        super().__init__(dir, ForwardDocument)
    def commit(self):
        self.whoosh_writer.commit()
        index = WhooshForwardIndex(self.dir)
        index.save(self.dir)

class WhooshPositionalBuilder(WhooshBuilder):
    def __init__(self, dir):
        super().__init__(dir, PositionalDocument)
    def commit(self):
        self.whoosh_writer.commit()
        index = WhooshPositionalIndex(self.dir)
        index.save(self.dir)

class WhooshIndex(Index):
    def __init__(self, dir):
        super().__init__(dir)
        self.whoosh_reader = whoosh.index.open_dir(dir).reader()    
    def total_freq(self, term):
        return self.whoosh_reader.frequency("content", term)
    def doc_freq(self, term):
        return self.whoosh_reader.doc_frequency("content", term)
    def doc_path(self, docid):
        return self.whoosh_reader.stored_fields(docid)['path']
    def ndocs(self):
        return self.whoosh_reader.doc_count()
    def all_terms(self):
        return list(self.whoosh_reader.field_terms("content"))
    def postings(self, term):
        return self.whoosh_reader.postings("content", term).items_as("frequency") \
            if self.doc_freq(term) > 0 else []

class WhooshForwardIndex(WhooshIndex):
    def term_freq(self, term, docID) -> int:
        if self.whoosh_reader.has_vector(docID, "content"):
            v = self.whoosh_reader.vector(docID, "content")
            v.skip_to(term)
            if v.id() == term:
                return v.value_as("frequency")
        return 0

class WhooshPositionalIndex(WhooshIndex):
    def positional_postings(self, term):
        return self.whoosh_reader.postings("content", term).items_as("positions") \
            if self.doc_freq(term) > 0 else []

class WhooshSearcher(Searcher):
    def __init__(self, dir):
        self.whoosh_index = whoosh.index.open_dir(dir)
        self.whoosh_searcher = self.whoosh_index.searcher()
        self.qparser = QueryParser("content", schema=self.whoosh_index.schema)
    def search(self, query, cutoff):
        return map(lambda scoredoc: (self.doc_path(scoredoc[0]), scoredoc[1]),
                   self.whoosh_searcher.search(self.qparser.parse(query), limit=cutoff).items())
    def doc_path(self, docid):
        return self.whoosh_index.reader().stored_fields(docid)['path']

## Calificación

Esta práctica se calificará con una puntuación de 0 a 10 atendiendo a las puntuaciones individuales de ejercicios y apartados dadas en el enunciado. No obstante, aquellos ejercicios marcados con un asterisco (*) tienen una complejidad un poco superior a los demás (que suman 7.5 puntos), y permiten, si se realizan todos, una nota superior a 10. 

El peso de la nota de esta práctica en la calificación final de prácticas es del **40%**.

La calificación se basará en a) el **número** de ejercicios realizados y b) la **calidad** de los mismos. La calidad se valorará por los **resultados** conseguidos (economía de consumo de RAM, disco y tiempo; tamaño de las colecciones que se consigan indexar) pero también del **mérito** en términos del interés de las técnicas aplicadas y la buena programación.

La puntuación que se indica en cada apartado es orientativa, en principio se aplicará tal cual se refleja pero podrá matizarse por criterios de buen sentido si se da el caso.

Para dar por válida la realización de un ejercicio, el código deberá funcionar (a la primera) integrado con las clases que se facilitan. El profesor comprobará este aspecto añadiendo los módulos entregados por el estudiante a los módulos facilitados en la práctica, ejecutando la función *main* así como otros main de prueba adicionales.

## Entrega

La entrega consistirá en un único fichero tipo *notebook* donde se incluirán todas las **implementaciones** solicitadas en cada ejercicio, así como una explicación de cada uno a modo de **memoria**.

## Indicaciones

Se sugiere trabajar en la práctica de manera incremental, asegurando la implementación de soluciones sencillas y mejorándolas de forma modular (la propia estructura de ejercicios plantea ya esta forma de trabajar).

Se podrán definir clases o módulos adicionales a las que se indican en el enunciado, por ejemplo, para reutilizar código. Y el estudiante podrá utilizar o no el software que se le proporciona, con la siguiente limitación: la función **main** deberá ejecutar correctamente <ins>sin ninguna modificación</ins> (más allá de comentar aquellos ejercicios que no se hayan realizado).

Asimismo, se recomienda indexar sin ningún tipo de stopwords ni stemming, para poder hacer pruebas más fácilmente con ejemplos “de juguete”.

# Ejercicio 1: Implementación de un modelo vectorial eficiente

Se mejorará la implementación de la práctica anterior aplicando algoritmos estudiados en las clases de teoría. En particular, se utilizarán listas de postings en lugar de un índice forward.

La reimplementación seguirá haciendo uso de la clase abstracta Index, y se podrá probar con cualquier implementación de esta clase (tanto la implementación de índice sobre Whoosh como las propias). 

## Ejercicio 1.1: Método orientado a términos (3pt)

Escribir una clase TermBasedVSMSearcher que implemente el modelo vectorial coseno por el método orientado a términos.

In [30]:
class TermBasedVSMSearcher(Searcher):
    def __init__(self, index, parser=BasicParser()):
        super().__init__(index, parser)
            
    def search(self, query, cutoff):
        qterms = self.parser.parse(query)
        
        scores = {}
        for term in qterms:
            postings = self.index.postings(term)
            for posting in postings:
                tf_value = tf(posting[1])
                idf_value = idf(self.index.doc_freq(term), self.index.ndocs())
                if posting[0] in scores.keys():
                    scores[posting[0]] += tf_value*idf_value
                else:
                    scores[posting[0]] = tf_value*idf_value
             
        ranking = SearchRanking(cutoff)
        for key, value in scores.items():
            ranking.push(self.index.doc_path(key), value / self.index.doc_module(key))
            
        return ranking

### Explicación/documentación

El método orientado a términos es una de las implementaciones populares que se pueden dar en el modelo vectorial. Consiste en una iteración secuencial sobre las listas de postings de cada término de la consulta. Como bien se puede ver en la implementación, estas son las dos iteraciones principales que se realizan. Cada posting de la lista contiene el doc_id del documento en el cual se encuentra el término y la frecuencia del término en el documento.

Procedemos a calcular el tf-idf de cada término de la consulta en cada documento. Si el doc_id del posting que estamos examinando ya está en el diccionario, simplemente acumulamos el tf-idf de ese término para ese doc_id correspondiente. Sin embargo, si el doc_id del posting no está, inicializamos ese posting con el tf-idf correspondiente. Una vez creado el diccionario con las listas de postings se procede a realizar el ranking. El Heap de Ranking se encarga de la ordenación y el devolver hasta un cutoff definido, en el método orientado a términos solo vamos introduciendo todos los cosenos que hayamos calculado y también los paths de los documentos.

## Ejercicio 1.2: Método orientado a documentos* (1pt)

Implementar el método orientado a documentos (con heap de postings) en una clase DocBasedVSMSearcher.

In [31]:
class DocBasedVSMSearcher(Searcher):
    # Your new code here (exercise 1.2*) #
    pass

### Explicación/documentación

(por hacer)

## Ejercicio 1.3: Heap de ránking (0.5pt)

Reimplementar la clase entregada SearchRanking para utilizar un heap de ránking (se recomienda usar el módulo [heapq](https://docs.python.org/3/library/heapq.html)). Nótese que esta opción se aprovecha mejor con la implementación orientada a documentos, aunque es compatible con la orientada a términos.

In [32]:
import heapq as heap
class SearchRanking:
    # TODO: to be implemented as heap (exercise 1.3) #
    def __init__(self, cutoff):
        self.ranking = [] # implementation as list, not as heap! TO BE MODIFIED
        self.cutoff = cutoff

    def push(self, docid, score):
        if self.ranking:
            if len(self.ranking) < self.cutoff:
                heap.heappush(self.ranking, (score, docid))
            else:
                min_elem = heap.heappop(self.ranking)

                if score > min_elem[0]:
                    heap.heappush(self.ranking, (score, docid))
                else:
                    heap.heappush(self.ranking, min_elem)
        else:
            heap.heappush(self.ranking, (score, docid))

    def __iter__(self):
        results = []
        len_heap = len(self.ranking)
        for num in range(self.cutoff):
            if num < len_heap:
                heap.heapify(self.ranking)
                results.append(heap.heappop(self.ranking))
            else:
                break
        
        sorted_results = sorted(results, key=lambda tup: tup[0], reverse=True)
        return iter(sorted_results)

### Explicación/documentación

Para la implementación de esta clase se ha empleado la librería heapq como se nos recomendaba en el enunciado.

La implementación ha constado de dos metodos, el método push que controla la inserción en el heap de ranking o no en función de si el score nuevo a introducir es mayor que el score minimo del heap o no (en el heap solo nos interesa guardar el top que nos indique cutoff hasta el momento). Y el metodo iter que reordena y saca los resultados finales del heap hasta que se vacíe.

# Ejercicio 2: Índice en RAM (3pt)

Implementar un índice propio que pueda hacer las mismas funciones que la implementación basada en Whoosh definida en la práctica 1. Como primera fase más sencilla, los índices se crearán completamente en RAM. Se guardarán a disco y leerán de disco en modo serializado (ver módulo [pickle](https://docs.python.org/3/library/pickle.html)).

Para guardar el índice se utilizarán los nombres de fichero definidos por las variables estáticas de la clase Config. 

Antes de guardar el índice, se borrarán todos los ficheros que pueda haber creados en el directorio del índice. Asimismo, el directorio se creará si no estuviera creado, de forma que no haga falta crearlo a mano. Este detalle se hará igual en los siguientes ejercicios.

## Ejercicio 2.1: Estructura de índice

Implementar la clase RAMIndex como subclase de Index con las estructuras necesarias: diccionario, listas de postings, más la información que se necesite. 

Para este ejercicio en las listas de postings sólo será necesario guardar los docIDs y las frecuencias; no es necesario almacenar las posiciones de los términos.

In [33]:
import pickle
class RAMIndex(Index):
    def __init__(self, index_path):
        self.dict_postings = {}
        super().__init__(index_path)
        self.index_path = index_path
    
    def postings(self, term):
        return self.dict_postings[term]
    
    def all_terms(self):
        return list(self.dict_postings.keys())
    
    def save(self, path):
        # guardamos lo basico de un indice
        super().save(path)
        
        # guardamos diccionario de postings
        p = os.path.join(path, Config.DICTIONARY_FILE)
        with open(p, 'wb') as f:
            pickle.dump(self.dict_postings, f) 
        
        # guardamos el fichero de paths
        p = os.path.join(path, Config.PATHS_FILE)
        with open(p, 'wb') as f:
            pickle.dump(self.docmap, f)
    
    def open(self, path):
        super().open(path)
        try:
            path_dict = os.path.join(path, Config.DICTIONARY_FILE)
            with open(path_dict, 'rb') as file_dict:
                self.dict_postings = pickle.load(file_dict)
                
            path_file = os.path.join(path, Config.PATHS_FILE)
            with open(path_file, 'rb') as file_paths:
                self.docmap = pickle.load(file_paths)
        except OSError:
            # the file may not exist the first time
            pass

### Explicación/documentación

Para la implementación del índice en RAM nos hemos basado en la implementación principal dada en las clases de teoría. Las claves del diccionario serán los términos y el valor de cada clave una lista que contiene tuplas con el siguiente formato (doc_id, frecuencia). Gracias a esta implementación, podemos sacar información del índice de forma bastante sencilla. La lista de postings para un término, será acceder al valor del diccionario mediante esa clave (término) y para obtener todos los términos devolvemos una lista con todas las claves del diccionario.

Hemos tenido que implementar los métodos de save y open ya que a parte de los módulos, en este índice se trabaja con la lista de postings que se crea y con el docmap que contiene los paths de los documentos. Por lo tanto, en cada método se guardan/abren los módulos con la llamada al constructor de la clase padre y luego guardamos/abrimos lo que hemos construido con el Builder. 

## Ejercicio 2.2 Construcción del índice

Implementar la clase RAMIndexBuilder como subclase de Builder, que cree todo el índice en RAM a partir de una colección de documentos.

In [34]:
import pickle
class RAMIndexBuilder(Builder):
    def __init__(self, path, parser=BasicParser()):
        if os.path.exists(path): shutil.rmtree(path)
        os.makedirs(path)
        self.path = path
        self.parser = parser
        self.index = RAMIndex(path)
    
    def index_document(self, path, text):
        # sacamos el id del documento a indexar (se comienza en 0)
        index_doc = self.index.ndocs()
        # añadimos este nuevo documento al indice
        self.index.add_doc(path)
        
        dterms = self.parser.parse(text)
        aux_dict = {}
        
        for term in dterms:
            if term in aux_dict.keys():
                aux_dict[term] += 1
            else:
                aux_dict[term] = 1
            
        for term in aux_dict.keys():
            if term in self.index.dict_postings.keys():
                self.index.dict_postings[term].append((index_doc, aux_dict[term]))
            else:
                self.index.dict_postings[term] = []
                self.index.dict_postings[term].append((index_doc, aux_dict[term]))
                
    def commit(self):
        self.index.save(self.path) 
        file = open(self.path + Config.INDEX_FILE, "wb")
        pickle.dump(self.index, file) 
        file.close()

### Explicación/documentación

Para esta implementación hemos desarrollado de nuevo los métodos de 'commit' e 'index_document'. En el commit simplemente guardamos el índice con el formato de fichero que se especifica en las variables estáticas de la clase Config. La librería pickle nos ayuda a guardar a disco en modo serializado. Para la indexación de un documento nuevo lo añadimos al docmap, creamos un diccionario auxiliar con las frecuencias de todos los términos, y por último, creamos el diccionario con las listas de postings. Cada clave será un término y el valor una lista de tuplas de formato: (doc_id, frecuencia).

# Ejercicio 3: Índice en disco* (1pt)

Reimplementar los índices definiendo las clases DiskIndex y DiskIndexBuilder de forma que:

*	El índice se siga creando entero en RAM (por ejemplo, usando estructuras similares a las del ejercicio 2).
*	Pero el índice se guarde en disco dato a dato (docIDs, frecuencias, etc.).
*	Al cargar el índice, sólo el diccionario se lee a RAM, y se accede a las listas de postings en disco cuando son necesarias (p.e. en tiempo de consulta).

Se sugiere guardar el diccionario en un fichero y las listas de postings en otro, utilizando los nombres de fichero definidos como variables estáticas en la clase Config.

Observación: se sugiere inicialmente guardar en disco las estructuras de índice en modo texto para poder depurar los programas. Una vez asegurada la corrección de los programas, puede ser más fácil pasar a modo binario o serializable (usando el módulo pickle como en ejercicios previos).

In [35]:
class DiskIndex(Index):
    
    def __init__(self, index_path):
        self.dict_postings = {}
        self.dict_terms = {}
        self.postings_path = os.path.join(index_path, Config.POSTINGS_FILE)
        super().__init__(index_path)
        self.index_path = index_path

    def postings(self, term):
        postings_list = []
        with open(self.postings_path, 'r') as f:
            info, postings = f.readline().strip().split("|")
            info = info.split(" ")
            postings = postings.split(" ")

            for i in range(0, int(info[1]) * 2, 2):
                postings_list.append([int(postings[i]), int(postings[i+1])])

        return postings_list

    def all_terms(self):
        return self.dict_postings.keys()

    def doc_freq(self, term):
        with open(self.postings_path, 'r') as f:
            f.seek(self.dict_postings[term])
            result = f.readline().split(" ")[1]
            
        return int(result[0])

    def save(self, path):
        
        with open(os.path.join(path, Config.PATHS_FILE), 'wb') as file_paths:
            pickle.dump(self.docmap, file_paths)

        with open(os.path.join(path, Config.POSTINGS_FILE), 'w') as file_postings:
            for term, postings in self.dict_postings.items():
                self.dict_postings[term] = file_postings.tell()
                n_postings = len(postings)
                file_postings.write(term + " " + str(n_postings) + "|")
                
                for p in postings:
                    file_postings.write(str(p[0]) + " ")
                    file_postings.write(str(p[1]) + " ")
                    if term not in self.dict_terms.keys():
                        self.dict_terms[term] = p[1]
                    else:
                        self.dict_terms[term] += p[1]
                
                file_postings.write("\n")

        with open(os.path.join(path, Config.DICTIONARY_FILE), 'wb') as file_dict:
            pickle.dump(self.dict_terms, file_dict)

        super().save(path)

    
    def open(self, path):
        super().open(path)
        try:
            with open(os.path.join(path, Config.PATHS_FILE), 'rb') as file_paths:
                self.docmap = pickle.load(file_paths)

            with open(os.path.join(path, Config.DICTIONARY_FILE), 'rb') as file_dict:
                self.dict_postings = pickle.load(file_dict)
        except OSError:
            # the file may not exist the first time
            pass



class DiskIndexBuilder(Builder):
    
    def __init__(self, path, parser=BasicParser()):
        if os.path.exists(path): shutil.rmtree(path)
        os.makedirs(path)
        self.path = path
        self.parser = parser
        self.index = DiskIndex(path)

    def index_document(self, path, text):
        # sacamos el id del documento a indexar (se comienza en 0)
        index_doc = self.index.ndocs()
        # añadimos este nuevo documento al indice
        self.index.add_doc(path)

        dterms = self.parser.parse(text)
        aux_dict = {}
        for term in dterms:
            if term in aux_dict.keys():
                aux_dict[term] += 1
            else:
                aux_dict[term] = 1
            
        for term in aux_dict.keys():
            if term in self.index.dict_postings.keys():
                self.index.dict_postings[term].append((index_doc, aux_dict[term]))
            else:
                self.index.dict_postings[term] = []
                self.index.dict_postings[term].append((index_doc, aux_dict[term]))
    
    
    def commit(self):
        self.index.save(self.path) 
        with open(self.path + Config.INDEX_FILE, "wb") as file_index:
            pickle.dump(self.index, file_index)

### Explicación/documentación

La implementación de esta clase ha sido muy similar a la del ejercicio anterior, la única diferencia es que ahora la lista de postings no se guarda en RAM si no que solo guardamos el diccionario con los terminos y sus frecuencias. Por tanto a la hora de obtener los postings accediendo al diccionario de postings, este no habrá sido previamente cargado de lo que teniamos en RAM sino del contenido en disco.

A partir de la colección de urls no llega a funcionar, pero para los ejemplos de toy, sí. En el main está comentado.

# Ejercicio 4: Motor de búsqueda proximal* (1pt)

Implementar un método de búsqueda proximal en una clase ProximitySearcher, utilizando las interfaces de índices posicionales. Igual que en los ejercicios anteriores, se sugiere definir esta clase como subclase (directa o indirecta) de Searcher. Para empezar a probar este buscador, se proporciona una implementación de indexación posicional basada en Whoosh (WhooshPositionalIndex).

In [36]:
class ProximitySearcher(Searcher):
    # Your new code here (exercise 4*) #
    pass

### Explicación/documentación

(por hacer)

# Ejercicio 5: Índice posicional* (1pt)

Implementar una variante adicional de índice (como subclase si se considera oportuno) que extienda las estructuras de índices con la inclusión de posiciones en las listas de postings. La implementación incluirá una clase PositionalIndexBuilder para la construcción del índice posicional así como una clase PositionalIndex para proporcionar acceso al mismo.

In [37]:
class PositionalIndex(RAMIndex):
    def __init__(self, path):
        self.dict_postings = {}
        super().__init__(path)
    
    def postings(self, term):
        return [(item[0], len(item[1])) for item in self.dict_postings[term]]
    
    def positional_postings(self, term):
        return [(item[0], item[1]) for item in self.dict_postings[term]]
            

class PositionalIndexBuilder(RAMIndexBuilder):
    def __init__(self, path, parser=BasicParser()):
        super().__init__(path)
        self.index = PositionalIndex(path)
        
    def index_document(self, path, text):
        # sacamos el id del documento a indexar (se comienza en 0)
        index_doc = self.index.ndocs()
        # añadimos este nuevo documento al indice
        self.index.add_doc(path)
        
        dterms = self.parser.parse(text)     
        aux_dict = {}
        for position, term in enumerate(dterms):
            if term in aux_dict.keys():
                aux_dict[term].append(position)
            else:
                aux_dict[term] = []
                aux_dict[term].append(position)
            
        for term in aux_dict.keys():
            if term in self.index.dict_postings.keys():
                self.index.dict_postings[term].append((index_doc, aux_dict[term]))
            else:
                self.index.dict_postings[term] = []
                self.index.dict_postings[term].append((index_doc, aux_dict[term]))   

### Explicación/documentación, indicando además el tipo de índice que se ha implementado y los aspectos que sean destacables

Hemos implementado el índice posicional como subclase de RAMIndex ya que en este índice se guarda la información en el mismo formato y lo que se guarda es muy parecido a lo implementado ya. Lo único que cambia del índice es que debemos implementar el método positional_postings para devolver la lista de postings de un término en el formato posicional (esto se explicará mas adelante) y también el método postings para devolver la lista en el formato tradicional: [(doc_id, frecuencia), (doc_id, frecuencia), ...].

Lo único que cambia al construir el índice sería la forma en la que se indexa un documento. En este caso, en vez de guardar la frecuencia, se va a guardar una lista con las posiciones en las que se encuentra el término dentro del documento. El formato dado un término sería el siguiente: 'aa' [(0, [0, 3, 9, 15]), (3, [6, 10, 16])]. Se crea un diccionario el cual cada clave contiene el término y el valor de la clave es una lista  de tuplas con el formato del ejemplo que se ha puesto; este formato explicado sería(doc_id, lista_posiciones): el término 'aa' se localiza en las posiciones 0, 3, 9 y 15 del documento con id = 0 y en las posiciones 6, 10 y 16 del documento con doc_id = 3.

# Ejercicio 6: PageRank (1pt)

Implementar el algoritmo PageRank en una clase PagerankDocScorer, que permitirá devolver un ranking de los documentos de manera similar a como hace un Searcher (pero sin recibir una consulta). 

Se recomienda, al menos inicialmente, llevar a cabo una implementación con la que los valores de PageRank sumen 1, para ayudar a la validación de la misma. Posteriormente, si se desea, se pueden escalar (o no, a criterio del estudiante) los cálculos omitiendo la división por el número total de páginas en el grafo. Será necesario tratar los nodos sumidero tal como se ha explicado en las clases de teoría.

In [38]:
import collections
class PagerankDocScorer():
    def __init__(self, graphfile, r, n_iter):
        self.r = r
        self.n_iter = n_iter
        file = open(graphfile, "r")
        lines = file.read().splitlines()
        self.pageRank_dict = {}
        for line in lines:
            links = line.split()
            if links[0] in self.pageRank_dict.keys():
                self.pageRank_dict[links[0]][2].append(links[1])
            else: 
                self.pageRank_dict[links[0]] = [0.0, 0.0, []]
                self.pageRank_dict[links[0]][2].append(links[1])
        
        list_sumideros = []
        for nodo in self.pageRank_dict.keys():
            for elem in self.pageRank_dict[nodo][2]:
                if elem not in self.pageRank_dict.keys():
                    list_sumideros.append(elem)
         
        for sumidero in list_sumideros:
            self.pageRank_dict[sumidero] = [0.0, 0.0, []]            
                    
        self.nodos = len(self.pageRank_dict.keys())
        for nodo in self.pageRank_dict.keys():
            self.pageRank_dict[nodo][0] = 1/self.nodos
        
        
    def rank(self, cutoff):
        for i in range(self.n_iter):
            for nodo1 in self.pageRank_dict.keys():
                nodos_from = []
                for nodo2 in self.pageRank_dict.keys():
                    if nodo1 in self.pageRank_dict[nodo2][2]:
                        nodos_from.append(nodo2)
                        
                self.pageRank_dict[nodo1][1] = self.pageRank_dict[nodo1][0]
                
                suma = 0
                for nodo in nodos_from: 
                    suma += self.pageRank_dict[nodo][1]/len(self.pageRank_dict[nodo][2])
                
                for nodo in self.pageRank_dict.keys():
                    if len(self.pageRank_dict[nodo][2]) == 0:
                        suma += self.pageRank_dict[nodo][1] / self.nodos 
                            
                
                self.pageRank_dict[nodo1][0] = (self.r/self.nodos) + (1-self.r)*suma
        
        sorted_list = sorted(self.pageRank_dict.items(), key=lambda item: item[1][0], reverse=True)
        ranking = [(elem[0], elem[1][0]) for elem in sorted_list[:cutoff]]
        return ranking

### Explicación/documentación

Para la implementación del PageRank hemos tomado la decisión de crear un diccionario donde cada clave se corresponde a cada nodo y asociado a cada clave se corresponde una lista de tres elementos donde el primer elemento guarda el valor del PageRank actual de ese nodo, el segundo elemento guarda el PageRank anterior de ese nodo y el último elemento guarda una lista con los enlaces salientes del nodo.

El método rank consiste en cumplir la función de cálculo del PageRank, la cual ha sido vista tanto en teoría como en las explicaciones de prácticas y también fijándonos en el algoritmo proporcionado en la explicación de la práctica.

El primer bucle nos indica el número de iteraciones a realizar el cálculo del PageRank ya que no se ha implementado la convergencia como mecanismo de parada. En los dos bucles posteriores calculamos la lista de nodos que tienen algún enlace saliente a cada nodo para poder asi obtener su PageRank. Por último aplicamos la fórmula con los datos ya obtenidos, ordenamos el diccionario en base al último score calculado, o lo que es lo mismo, score actual(primera posición en la lista) y devolvemos los resultados en forma de lista.

# Programa de prueba **main**

Descarga los ficheros del curso de Moodle y coloca sus contenidos en una carpeta **collections** en el mismo directorio que este *notebook*. El fichero <u>toys.zip</u> hay que descomprimirlo para indexar las carpetas que contiene. Igualmente, el fichero <u>graphs.zip</u> incluye ficheros (*1k-links.dat*, *toy-graph1.dat*, *toy-graph2.dat*) que se deben descomprimir en la carpeta collections para que el main funcione.

## Función **main**

In [39]:
import os
import shutil
import time

def main():
    index_root_dir = "./index/"
    collections_root_dir = "./collections/"
    test_collection (collections_root_dir + "toys/toy1/", index_root_dir + "toys/toy1/", "cc", ["aa dd", "aa"], False)
    test_collection (collections_root_dir + "toys/toy2/", index_root_dir + "toys/toy2/", "aa", ["aa cc", "bb aa"], False)
    test_collection (collections_root_dir + "urls.txt", index_root_dir + "urls/", "wikipedia", ["information probability", "probability information", "higher probability"], True)
    test_collection (collections_root_dir + "docs1k.zip", index_root_dir + "docs1k/", "seat", ["obama family tree"], True)
    test_collection (collections_root_dir + "docs10k.zip", index_root_dir + "docs10k/", "seat", ["obama family tree"], False)
    test_pagerank("./collections/", 5)

def test_collection(collection_path: str, index_path: str, word: str, queries: list, analyse_performance: bool):
    print("=================================================================")
    print("Testing indices and search on " + collection_path)

    # We now test building different implementations of an index
    test_build(WhooshBuilder(index_path + "whoosh"), collection_path)
    test_build(WhooshForwardBuilder(index_path + "whoosh_fwd"), collection_path)
    test_build(WhooshPositionalBuilder(index_path + "whoosh_pos"), collection_path)
    test_build(RAMIndexBuilder(index_path + "ram"), collection_path)
    test_build(DiskIndexBuilder(index_path + "disk"), collection_path)
    test_build(PositionalIndexBuilder(index_path + "pos"), collection_path)

    # We now inspect all the implementations
    indices = [
            WhooshIndex(index_path + "whoosh"),
            WhooshForwardIndex(index_path + "whoosh_fwd"), 
            WhooshPositionalIndex(index_path + "whoosh_pos"), 
            RAMIndex(index_path + "ram"),
            # DiskIndex(index_path + "disk"),
            PositionalIndex(index_path + "pos"),
            ]
    for index in indices:
        test_read(index, word)

    for query in queries:
        print("------------------------------")
        print("Checking search results for %s" % (query))
        # Whoosh searcher can only work with its own indices
        test_search(WhooshSearcher(index_path + "whoosh"), WhooshIndex(index_path + "whoosh"), query, 5)
        test_search(WhooshSearcher(index_path + "whoosh_fwd"), WhooshForwardIndex(index_path + "whoosh_fwd"), query, 5)
        test_search(WhooshSearcher(index_path + "whoosh_pos"), WhooshPositionalIndex(index_path + "whoosh_pos"), query, 5)
        # test_search(ProximitySearcher(WhooshPositionalIndex(index_path + "whoosh_pos")), WhooshPositionalIndex(index_path + "whoosh_pos"), query, 5)
        for index in indices:
            # our searchers should work with any other index
            test_search(SlowVSMSearcher(index), index, query, 5)
            test_search(TermBasedVSMSearcher(index), index, query, 5)
            # test_search(DocBasedVSMSearcher(index), index, query, 5)
            # test_search(ProximitySearcher(PositionalIndex(index_path + "pos")), PositionalIndex(index_path + "pos"), query, 5)

    # if we keep the list in memory, there may be problems with accessing the same index twice
    indices = list()

    if analyse_performance:
        # let's analyse index performance
        test_index_performance(collection_path, index_path)
        # let's analyse search performance
        for query in queries:
            test_search_performance(collection_path, index_path, query, 5)

def test_build(builder, collection):
    stamp = time.time()
    print("Building index with", type(builder))
    print("Collection:", collection)
    # this function should index the recieved collection and add it to the index
    builder.build(collection)
    # when we commit, the information in the index becomes persistent
    # we can also save any extra information we may need
    # (and that cannot be computed until the entire collection is scanned/indexed)
    builder.commit()
    print("Done (", time.time() - stamp, "seconds )")
    print()

def test_read(index, word):
    stamp = time.time()
    print("Reading index with", type(index))
    print("Collection size:", index.ndocs())
    print("Vocabulary size:", len(index.all_terms()))
    # more tests
    doc_id = 0
    print("  Frequency of word \"" + word + "\" in document " + str(doc_id) + " - " + index.doc_path(doc_id) + ": " + str(index.term_freq(word, doc_id)))
    print("  Total frequency of word \"" + word + "\" in the collection: " + str(index.total_freq(word)) + " occurrences over " + str(index.doc_freq(word)) + " documents")
    print("  Docs containing the word '" + word + "':", index.doc_freq(word))
    print("    First two documents:", [(doc, freq) for doc, freq in index.postings(word)][0:2])
    print("Done (", time.time() - stamp, "seconds )")
    print()


def test_search (engine, index, query, cutoff):
    stamp = time.time()
    print("  " + engine.__class__.__name__ + " with index " + index.__class__.__name__ + " for query '" + query + "'")
    for path, score in engine.search(query, cutoff):
        print(score, "\t", path)
    print()
    print("Done (", time.time() - stamp, "seconds )")
    print()

def disk_space(index_path: str) -> int:
    space = 0
    for f in os.listdir(index_path):
        p = os.path.join(index_path, f)
        if os.path.isfile(p):
            space += os.path.getsize(p)
    return space

def test_index_performance (collection_path: str, base_index_path: str):
    print("----------------------------")
    print("Testing index performance on " + collection_path + " document collection")

    print("  Build time...")
    start_time = time.time()
    b = WhooshBuilder(base_index_path + "whoosh")
    b.build(collection_path)
    b.commit()
    print("\tWhooshIndex: %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    b = WhooshForwardBuilder(base_index_path + "whoosh_fwd")
    b.build(collection_path)
    b.commit()
    print("\tWhooshForwardIndex: %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    b = WhooshPositionalBuilder(base_index_path + "whoosh_pos")
    b.build(collection_path)
    b.commit()
    print("\tWhooshPositionalIndex: %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    b = RAMIndexBuilder(base_index_path + "ram")
    b.build(collection_path)
    b.commit()
    print("\tRAMIndex: %s seconds ---" % (time.time() - start_time))
    # start_time = time.time()
    # b = DiskIndexBuilder(base_index_path + "disk")
    # b.build(collection_path)
    # b.commit()
    # print("\tDiskIndex: %s seconds ---" % (time.time() - start_time))

    print("  Load time...")
    start_time = time.time()
    WhooshIndex(base_index_path + "whoosh")
    print("\tWhooshIndex: %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    WhooshForwardIndex(base_index_path + "whoosh_fwd")
    print("\tWhooshForwardIndex: %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    WhooshPositionalIndex(base_index_path + "whoosh_pos")
    print("\tWhooshPositionalIndex: %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    RAMIndex(base_index_path + "ram")
    print("\tRAMIndex: %s seconds ---" % (time.time() - start_time))
    # start_time = time.time()
    # DiskIndex(base_index_path + "disk")
    # print("\tDiskIndex: %s seconds ---" % (time.time() - start_time))

    print("  Disk space...")
    print("\tWhooshIndex: %s space ---" % (disk_space(base_index_path + "whoosh")))
    print("\tWhooshForwardIndex: %s space ---" % (disk_space(base_index_path + "whoosh_fwd")))
    print("\tWhooshPositionalIndex: %s space ---" % (disk_space(base_index_path + "whoosh_pos")))
    print("\tRAMIndex: %s space ---" % (disk_space(base_index_path + "ram")))
    # print("\tDiskIndex: %s space ---" % (disk_space(base_index_path + "disk")))


def test_search_performance (collection_name: str, base_index_path: str, query: str, cutoff: int):
    print("----------------------------")
    print("Testing search performance on " + collection_name + " document collection with query: '" + query + "'")
    whoosh_index = WhooshIndex(base_index_path + "whoosh")
    ram_index = RAMIndex(base_index_path + "ram")
    # disk_index = DiskIndex(base_index_path + "disk")

    start_time = time.time()
    test_search(WhooshSearcher(base_index_path + "whoosh"), whoosh_index, query, cutoff)
    print("--- Whoosh on Whoosh %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    test_search(SlowVSMSearcher(whoosh_index), whoosh_index, query, cutoff)
    print("--- SlowVSM on Whoosh %s seconds ---" % (time.time() - start_time))

    # let's test some combinations of ranking + index implementations
    start_time = time.time()
    test_search(TermBasedVSMSearcher(whoosh_index), whoosh_index, query, cutoff)
    print("--- TermVSM on Whoosh %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    test_search(TermBasedVSMSearcher(ram_index), ram_index, query, cutoff)
    print("--- TermVSM on RAM %s seconds ---" % (time.time() - start_time))
    #start_time = time.time()
    #test_search(TermBasedVSMSearcher(disk_index), disk_index, query, cutoff)
    #print("--- TermVSM on Disk %s seconds ---" % (time.time() - start_time))

    #start_time = time.time()
    #test_search(DocBasedVSMSearcher(disk_index), disk_index, query, cutoff)
    #print("--- DocVSM on Disk %s seconds ---" % (time.time() - start_time))

def test_pagerank(graphs_root_dir, cutoff):
    print("----------------------------")
    print("Testing PageRank")
    # we separate this function because it cannot work with all the collections
    start_time = time.time()
    for path, score in PagerankDocScorer(graphs_root_dir + "toy-graph1.dat", 0.5, 50).rank(cutoff):
        print(score, "\t", path)
    print()
    print("--- Pagerank with toy_graph_1 %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    for path, score in PagerankDocScorer(graphs_root_dir + "toy-graph2.dat", 0.6, 50).rank(cutoff):
        print(score, "\t", path)
    print()
    print("--- Pagerank with toy_graph_2 %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    for path, score in PagerankDocScorer(graphs_root_dir + "1k-links.dat", 0.2, 50).rank(cutoff):
        print(score, "\t", path)
    print()
    print("--- Pagerank with simulated links for doc1k %s seconds ---" % (time.time() - start_time))

main()


Testing indices and search on ./collections/toys/toy1/
Building index with <class '__main__.WhooshBuilder'>
Collection: ./collections/toys/toy1/
Done ( 0.019369125366210938 seconds )

Building index with <class '__main__.WhooshForwardBuilder'>
Collection: ./collections/toys/toy1/
Done ( 0.010722637176513672 seconds )

Building index with <class '__main__.WhooshPositionalBuilder'>
Collection: ./collections/toys/toy1/
Done ( 0.008970022201538086 seconds )

Building index with <class '__main__.RAMIndexBuilder'>
Collection: ./collections/toys/toy1/
Done ( 0.0008933544158935547 seconds )

Building index with <class '__main__.DiskIndexBuilder'>
Collection: ./collections/toys/toy1/
Done ( 0.0035543441772460938 seconds )

Building index with <class '__main__.PositionalIndexBuilder'>
Collection: ./collections/toys/toy1/
Done ( 0.0009160041809082031 seconds )

Reading index with <class '__main__.WhooshIndex'>
Collection size: 4
Vocabulary size: 39
  Frequency of word "cc" in document 0 - ./colle

Done ( 3.3373589515686035 seconds )

Building index with <class '__main__.WhooshForwardBuilder'>
Collection: ./collections/urls.txt
Done ( 3.0036323070526123 seconds )

Building index with <class '__main__.WhooshPositionalBuilder'>
Collection: ./collections/urls.txt
Done ( 2.665405035018921 seconds )

Building index with <class '__main__.RAMIndexBuilder'>
Collection: ./collections/urls.txt
Done ( 1.8450298309326172 seconds )

Building index with <class '__main__.DiskIndexBuilder'>
Collection: ./collections/urls.txt
Done ( 2.1475260257720947 seconds )

Building index with <class '__main__.PositionalIndexBuilder'>
Collection: ./collections/urls.txt
Done ( 1.8579277992248535 seconds )

Reading index with <class '__main__.WhooshIndex'>
Collection size: 3
Vocabulary size: 5899
  Frequency of word "wikipedia" in document 0 - https://en.wikipedia.org/wiki/Simpson's_paradox: 5
  Total frequency of word "wikipedia" in the collection: 23.0 occurrences over 3 documents
  Docs containing the word 

	WhooshIndex: 2.3596839904785156 seconds ---
	WhooshForwardIndex: 2.671544075012207 seconds ---
	WhooshPositionalIndex: 2.8233237266540527 seconds ---
	RAMIndex: 1.970839262008667 seconds ---
  Load time...
	WhooshIndex: 0.0013935565948486328 seconds ---
	WhooshForwardIndex: 0.0010342597961425781 seconds ---
	WhooshPositionalIndex: 0.0010662078857421875 seconds ---
	RAMIndex: 0.003216266632080078 seconds ---
  Disk space...
	WhooshIndex: 1060414 space ---
	WhooshForwardIndex: 1134673 space ---
	WhooshPositionalIndex: 1212449 space ---
	RAMIndex: 123609 space ---
----------------------------
Testing search performance on ./collections/urls.txt document collection with query: 'information probability'
  WhooshSearcher with index WhooshIndex for query 'information probability'
2.9288499170082907 	 https://en.wikipedia.org/wiki/Entropy
2.4707338357114654 	 https://en.wikipedia.org/wiki/Simpson's_paradox
2.087930188843972 	 https://en.wikipedia.org/wiki/Bias

Done ( 0.0037107467651367188 se

15.808952372971135 	 clueweb09-en0001-02-21241.html
15.611476287511962 	 clueweb09-en0008-45-29117.html
15.554087101921635 	 clueweb09-enwp01-59-16163.html

Done ( 0.05137205123901367 seconds )

  SlowVSMSearcher with index WhooshIndex for query 'obama family tree'
clueweb09-en0010-79-2218.html 	 0.28421789364376177
clueweb09-en0009-30-2768.html 	 0.22631966869496392
clueweb09-en0001-02-21241.html 	 0.224801913947326
clueweb09-en0009-30-2441.html 	 0.22386305596546352
clueweb09-en0009-30-2755.html 	 0.22349064069479563

Done ( 1.9562327861785889 seconds )

  TermBasedVSMSearcher with index WhooshIndex for query 'obama family tree'
clueweb09-en0010-79-2218.html 	 0.28421789364376177
clueweb09-en0009-30-2768.html 	 0.22631966869496392
clueweb09-en0001-02-21241.html 	 0.224801913947326
clueweb09-en0009-30-2441.html 	 0.22386305596546352
clueweb09-en0009-30-2755.html 	 0.22349064069479563

Done ( 0.012063980102539062 seconds )

  SlowVSMSearcher with index WhooshForwardIndex for query 'oba

KeyboardInterrupt: 

### Resumen de coste y rendimiento

Hay que analizar las **diferencias de rendimiento** observadas entre las diferentes implementaciones que se han creado y probado para cada componente.

En concreto, hay que reportar tiempo de indexado, consumo máximo de RAM y espacio en disco al construir el índice, y el tiempo de carga y consumo máximo de RAM al cargar el índice para cada una de las colecciones utilizadas.

Por ejemplo:

|      | Construcción | del | índice | Carga del | índice |
|------|--------------------|-----------------|------------------|-----------------|-----------------|
|      | Tiempo de indexado | Consumo máx RAM | Espacio en disco | Tiempo de carga | Consumo máx RAM |
| toy1 | | | | | |
| toy2 | | | | | |
| 1K | | | | | |
| 10K | | | | | |
